# Gaussian experiment
All data is generated on the fly

In [1]:
import sys
sys.path.append('../python')

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

from machine_learning import *
import scipy.stats
import os
import sobol
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""

/usr/lib64/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Setup

In [2]:
def generate_sobol_points(M, dim):
    points = []
    for i in range(M):
        points.append(sobol.i4_sobol(dim,i)[0])
    return np.array(points)

dim = 6
M = int(2**10)

data_sources = {"Monte Carlo": np.random.uniform(0, 1, (M,dim)),
                "QMC Sobol" : generate_sobol_points(M, dim)}

def sine_functional(x):
    return np.sum(np.sin(4*np.pi*x), 1)
def normal_functional(x):
    return scipy.stats.norm.ppf(x)

def sum_functional(x):
    return np.sum(x, 1)

functionals = {"Sum": sum_functional,
               "Sine" : sine_functional,
               "Sine Normal" : lambda x : sine_functional(normal_functional(x)),
               "Sum Normal" : lambda x: sum_functional(normal_functional(x))
               }


gaussian_network =  [12, 12, 10, 12, 10, 12, 10, 10, 10, 12, 12,1]

epochs = 500000



# Training setup

In [3]:
def train(*, parameters, samples, title):
    train_sizes = [ 128, 256, 512, 1024]
    
    optimizers = {"SGD": keras.optimizers.SGD,
                 "Adam": keras.optimizers.Adam}
    
    losses = ["mean_squared_error", "mean_absolute_error"]
    
    for optimizer in optimizers.keys():
        for loss in losses:
            display(HTML("<h1>{}: Computed with {} with {}</h1>".format(title, optimizer, loss)))
    
    
            for train_size in train_sizes:
                tables = Tables.make_default()
                batch_size = train_size
                validation_size=train_size
                
                
                
                network_information = NetworkInformation(optimizer=optimizers[optimizer], epochs=epochs, 
                                                             network=gaussian_network, train_size=train_size,
                                                             validation_size=validation_size,
                                                            loss=loss)
                    
                output_information = OutputInformation(tables=tables, title=title,
                                                          short_title=title)
                showAndSave.prefix='%s_%s_%s_ts_%d_bs_%d' %(title, optimizer, loss, batch_size, train_size)
                get_network_and_postprocess(parameters, samples, network_information = network_information,
                        output_information = output_information)
                
                showAndSave.prefix='%s_%s_%s_all_ts_%d_bs_%d' %(title, optimizer, loss, batch_size, train_size)
                tables.write_tables()
            
    

# Training

In [ ]:
for data_source_name in data_sources.keys():
   
    for functional_name in functionals.keys():
        parameters = data_sources[data_source_name]
        functional = functionals[functional_name]
        
        samples = functional(parameters)
        title = '{} with {}'.format(functional_name, data_source_name)
        
        train(parameters=parameters, samples=samples, title=title)
   